<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>



# Regresión Lineal Múltiple 

Tiempo estimado necesario: **15** minutos

-----

## Objetivos 

Tras completar este laboratorio, podrás:

  * Utilizar scikit-learn para implementar la **Regresión Lineal Múltiple**.
  * Crear un modelo, entrenarlo, probarlo y usar el modelo.



<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#understanding-data">Entendiendo los Datos</a></li>
        <li><a href="#reading_data">Lectura de los Datos</a></li>
        <li><a href="#multiple_regression_model">Modelo de Regresión Múltiple</a></li>
        <li><a href="#prediction">Predicción</a></li>
        <li><a href="#practice">Práctica</a></li>
    </ol>
</div>
<br>
<hr>


### Importación de los Paquetes Necesarios


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

### Descarga de Datos

Para descargar los datos, utilizaremos `!wget` para bajarlos desde IBM Object Storage.


In [ ]:
!wget -O FuelConsumption.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv


<h2 id="understanding_data">Entendiendo los Datos</h2>

### `FuelConsumption.csv`:

Hemos descargado un conjunto de datos de consumo de combustible, **`FuelConsumption.csv`**, que contiene clasificaciones de consumo de combustible específicas del modelo y emisiones estimadas de dióxido de carbono para vehículos nuevos de servicio liviano para la venta al por menor en Canadá. [Fuente del conjunto de datos](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

  * **MODELYEAR** (AÑO DEL MODELO) p.ej. 2014
  * **MAKE** (MARCA) p.ej. Acura
  * **MODEL** (MODELO) p.ej. ILX
  * **VEHICLE CLASS** (CLASE DE VEHÍCULO) p.ej. SUV
  * **ENGINE SIZE** (TAMAÑO DEL MOTOR) p.ej. 4.7
  * **CYLINDERS** (CILINDROS) p.ej 6
  * **TRANSMISSION** (TRANSMISIÓN) p.ej. A6
  * **FUELTYPE** (TIPO DE COMBUSTIBLE) p.ej. z
  * **FUEL CONSUMPTION in CITY(L/100 km)** (CONSUMO EN CIUDAD (L/100 km)) p.ej. 9.9
  * **FUEL CONSUMPTION in HWY (L/100 km)** (CONSUMO EN CARRETERA (L/100 km)) p.ej. 8.9
  * **FUEL CONSUMPTION COMB (L/100 km)** (CONSUMO COMBINADO (L/100 km)) p.ej. 9.2
  * **CO2 EMISSIONS (g/km)** (EMISIONES DE CO2 (g/km)) p.ej. 182   --\> bajo --\> 0



<h2 id="reading_data">Lectura de los Datos</h2>


In [ ]:
df = pd.read_csv("FuelConsumption.csv")

# take a look at the dataset
df.head()

Seleccionemos algunas características que queremos usar para la regresión.


In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head(9)

Grafiquemos los valores de Emisión con respecto al tamaño del motor:


In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Creación del Conjunto de Datos de Entrenamiento y Prueba

La División de Entrenamiento/Prueba (*Train/Test Split*) implica dividir el conjunto de datos en conjuntos de entrenamiento y prueba respectivamente, que son **mutuamente excluyentes**. Después, se entrena con el conjunto de entrenamiento y se prueba con el conjunto de prueba.
Esto proporcionará una evaluación más precisa de la **precisión fuera de la muestra** (*out-of-sample accuracy*), ya que el conjunto de datos de prueba no forma parte del conjunto de datos que se ha utilizado para entrenar el modelo. Por lo tanto, nos da una mejor comprensión de qué tan bien nuestro modelo **generaliza** a nuevos datos.

Conocemos el resultado de cada punto de datos en el conjunto de prueba, ¡lo que lo hace ideal para probar\! Dado que estos datos no se han utilizado para entrenar el modelo, el modelo no tiene conocimiento del resultado de estos puntos de datos. Así que, en esencia, es verdaderamente una prueba fuera de la muestra.

Dividamos nuestro conjunto de datos en conjuntos de entrenamiento y prueba. Alrededor del **80%** del conjunto de datos completo se utilizará para el entrenamiento y el **20%** para las pruebas. Creamos una máscara para seleccionar filas aleatorias utilizando la función **`np.random.rand()`**:


In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

#### Distribución de los Datos de Entrenamiento


In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

<h2 id="multiple_regression_model">Modelo de Regresión Múltiple</h2>


En realidad, existen múltiples variables que impactan la emisión de CO2. Cuando está presente **más de una variable independiente**, el proceso se denomina **Regresión Lineal Múltiple**. Un ejemplo de regresión lineal múltiple es predecir la emisión de CO2 utilizando las características **FUELCONSUMPTION\_COMB**, **EngineSize** y **Cylinders** de los coches. Lo bueno aquí es que el modelo de regresión lineal múltiple es la extensión del modelo de regresión lineal simple.


In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y = np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Como se mencionó antes, el **Coeficiente** y el **Intercepto** son los parámetros de la línea ajustada.
Dado que es un modelo de regresión lineal múltiple con 3 parámetros y que los parámetros son el intercepto y los coeficientes del hiperplano, sklearn puede estimarlos a partir de nuestros datos. Scikit-learn utiliza el método simple de **Mínimos Cuadrados Ordinarios (OLS)** para resolver este problema.

#### Mínimos Cuadrados Ordinarios (OLS)

OLS es un método para estimar los parámetros desconocidos en un modelo de regresión lineal. OLS elige los parámetros de una función lineal de un conjunto de variables explicativas **minimizando la suma de los cuadrados de las diferencias** entre la variable dependiente objetivo y las predichas por la función lineal. En otras palabras, intenta minimizar la Suma de Errores Cuadrados (**SSE**) o el Error Cuadrático Medio (**MSE**) entre la variable objetivo ($y$) y nuestra salida predicha ($\hat{y}$) sobre todas las muestras en el conjunto de datos.

OLS puede encontrar los mejores parámetros utilizando uno de los siguientes métodos:

  * Resolver los parámetros del modelo analíticamente utilizando ecuaciones de **forma cerrada**.
  * Utilizar un **algoritmo de optimización** (Descenso de Gradiente, Descenso de Gradiente Estocástico, Método de Newton, etc.).


<h2 id="prediction">Predicción</h2>


In [ ]:
y_hat= regr.predict(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
x = np.asanyarray(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y = np.asanyarray(test[['CO2EMISSIONS']])
print("Mean Squared Error (MSE) : %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

**Puntuación de Regresión de Varianza Explicada:**
Sea $\hat{y}$ el resultado objetivo estimado, $y$ el resultado objetivo correspondiente (correcto), y $Var$ la Varianza (el cuadrado de la desviación estándar). Entonces, la varianza explicada se estima de la siguiente manera:

$$\texttt{explainedVariance}(y, \hat{y}) = 1 - \frac{Var\{ y - \hat{y}\}}{Var\{y\}}$$

La mejor puntuación posible es **1.0**, y los valores más bajos son peores.


<h2 id="practice">Practica</h2>

Intenta usar una regresión lineal múltiple con el mismo conjunto de datos, pero esta vez usa **FUELCONSUMPTION_CITY** y **FUELCONSUMPTION_HWY** en lugar de **FUELCONSUMPTION_COMB**. 

¿Resulta en una mejor precisión?

In [ ]:
# Importa el modelo de regresión lineal desde scikit-learn

# Selecciona las columnas que usarás como variables independientes (X) del conjunto de entrenamiento.
# asegúrate de que las columnas existen en el DataFrame 'train' y que están bien escritas.

# Selecciona la variable dependiente (Y), es decir, el valor que queremos predecir: las emisiones de CO2.

# Entrena el modelo con los datos de entrenamiento (ajusta los coeficientes).

# Muestra los coeficientes obtenidos tras el entrenamiento.
# cada número indica cuánto influye cada variable (motor, cilindros, consumo...) en las emisiones.

# Realiza predicciones con el conjunto de prueba usando las mismas variables independientes.
# cambia las columnas si tu conjunto de datos tiene nombres diferentes.

# Extrae las variables reales (X e Y) del conjunto de prueba para poder evaluar el modelo.

# Calcula el error cuadrático medio (MSE), una medida de la precisión del modelo.
# cuanto menor sea este valor, mejor es el ajuste.

# Calcula el coeficiente de determinación R² (score del modelo).
#  un valor cercano a 1 indica que el modelo explica bien la variabilidad de los datos.



### ¡Gracias por completar este laboratorio! 🎉

## Autor

Saeed Aghabozorgi

### Otros Contribuidores

<a href="https://www.linkedin.com/in/joseph-s-50398b136/" target="_blank">Joseph Santarcangelo</a>

### Traducción

<a href="https://www.linkedin.com/in/carlostessier/" target="_blank">Carlos Tessier</a>

 

## <h3 align="center"> © Corporación IBM 2025. Todos los derechos reservados. <h3/>